# Data Cleaning File

## Setup
Imports, reading in files, etc..

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

from tqdm import tqdm
import zlib

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rc
from matplotlib.ticker import PercentFormatter

In [2]:

#https://realpython.com/python-zip-function/#:~:text=%20Using%20the%20Python%20zip%20()%20Function%20for,zip%20()%20function%20works%20differently%20in...%20More

df, git_path = {}, '../data/'
for file_name, file_ref, n, t in zip(['test.csv', 'training.csv', 'IdLookupTable.csv', 'SampleSubmission.csv'],
                        ['test', 'train', 'id_lookup', 'sample_submission', ],
                        [   #test
                            ['image_id', 'image'], 
                            #train
                            ['left_eye_center_x', 'left_eye_center_y',  
                            'right_eye_center_x', 'right_eye_center_y', 
                            'left_eye_inner_corner_x', 'left_eye_inner_corner_y', 
                            'left_eye_outer_corner_x', 'left_eye_outer_corner_y', 
                            'right_eye_inner_corner_x', 'right_eye_inner_corner_y', 
                            'right_eye_outer_corner_x', 'right_eye_outer_corner_y', 
                            'left_eyebrow_inner_end_x', 'left_eyebrow_inner_end_y', 
                            'left_eyebrow_outer_end_x', 'left_eyebrow_outer_end_y', 
                            'right_eyebrow_inner_end_x', 'right_eyebrow_inner_end_y', 
                            'right_eyebrow_outer_end_x', 'right_eyebrow_outer_end_y', 
                            'nose_tip_x', 'nose_tip_y', 
                            'mouth_left_corner_x', 'mouth_left_corner_y', 
                            'mouth_right_corner_x', 'mouth_right_corner_y', 
                            'mouth_center_top_lip_x', 'mouth_center_top_lip_y', 
                            'mouth_center_bottom_lip_x', 'mouth_center_bottom_lip_y', 'image'],
                            #IdLookupTable
                            ['row_id', 'image_id', 'feature_name', 'location'],
                            #SampleSubmission
                            ['row_id', 'location']
                        ],

                        [
                             #test   
                            {'image_id':'uint16', 'image':'object'},
                            #train
                            {'left_eye_center_x':'float32', 'left_eye_center_y':'float32', 
                            'right_eye_center_x':'float32', 'right_eye_center_y':'float32', 
                            'left_eye_inner_corner_x':'float32', 'left_eye_inner_corner_y':'float32', 
                            'left_eye_outer_corner_x':'float32', 'left_eye_outer_corner_y':'float32', 
                            'right_eye_inner_corner_x':'float32', 'right_eye_inner_corner_y':'float32',
                            'right_eye_outer_corner_x':'float32', 'right_eye_outer_corner_y':'float32', 
                            'left_eyebrow_inner_end_x':'float32', 'left_eyebrow_inner_end_y':'float32',
                            'left_eyebrow_outer_end_x':'float32', 'left_eyebrow_outer_end_y':'float32', 
                            'right_eyebrow_inner_end_x':'float32', 'right_eyebrow_inner_end_y':'float32',
                            'right_eyebrow_outer_end_x':'float32', 'right_eyebrow_outer_end_y':'float32', 
                            'nose_tip_x':'float32', 'nose_tip_y':'float32', 'mouth_left_corner_x':'float32',
                            'mouth_left_corner_y':'float32', 'mouth_right_corner_x':'float32', 
                            'mouth_right_corner_y':'float32', 'mouth_center_top_lip_x':'float32', 
                            'mouth_center_top_lip_y':'float32','mouth_center_bottom_lip_x':'float32', 
                            'mouth_center_bottom_lip_y':'float32', 'image':'object'},
                             #IdLookupTable
                            {'row_id':'uint16', 'image_id':'uint16', 'location':'float32'},
                            #SampleSubmission
                            {'row_id':'uint16', 'location':'float32'}
                        ],
                        ):
    #This is the begining of the for loop for each file:
    print("Load files.")
    print("Begin loading file '%s' " % "".join( (git_path, file_name)))
    #print(file_ref)
    df[file_ref] = pd.read_csv("".join( (git_path,file_name) ), names = n, dtype = t, skiprows = 1)
    
    #If the file contains an image column like in the case of test.csv store those images now. 
    if "image" in df[file_ref]:
        print("\tFound %d images. Processing. " % df[file_ref].shape[0])
        #Get the row with the image data and store it in the dataframe 
        df[file_ref]['image'] = df[file_ref]["image"].map(lambda x: np.array(list(map(int, x.split(" ")))))
    print("\tFile", file_ref, " with shape:", df[file_ref].shape, " load complete\n")

print("Load files complete")

Load files.
Begin loading file '../data/test.csv' 
	Found 1783 images. Processing. 
	File test  with shape: (1783, 2)  load complete

Load files.
Begin loading file '../data/training.csv' 
	Found 7049 images. Processing. 
	File train  with shape: (7049, 31)  load complete

Load files.
Begin loading file '../data/IdLookupTable.csv' 
	File id_lookup  with shape: (27124, 4)  load complete

Load files.
Begin loading file '../data/SampleSubmission.csv' 
	File sample_submission  with shape: (27124, 2)  load complete

Load files complete


In [7]:
#Creating a copy of the train data in train_data in case you want to add columns back in from df[train]
train_data=df['train'].copy(deep=True)
train_data

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,image
0,66.033562,39.002274,30.227007,36.421677,59.582077,39.647423,73.130348,39.969997,36.356571,37.389400,...,57.066803,61.195309,79.970169,28.614496,77.388992,43.312603,72.935455,43.130707,84.485771,"[238, 236, 237, 238, 240, 240, 239, 241, 241, ..."
1,64.332939,34.970078,29.949276,33.448715,58.856171,35.274349,70.722725,36.187164,36.034725,34.361530,...,55.660934,56.421448,76.351997,35.122383,76.047661,46.684597,70.266556,45.467915,85.480171,"[219, 215, 204, 196, 204, 211, 212, 200, 180, ..."
2,65.057053,34.909641,30.903790,34.909641,59.411999,36.320969,70.984421,36.320969,37.678104,36.320969,...,53.538948,60.822948,73.014313,33.726315,72.732002,47.274948,70.191788,47.274948,78.659370,"[144, 142, 159, 180, 188, 188, 184, 180, 167, ..."
3,65.225739,37.261772,32.023094,37.261772,60.003338,39.127178,72.314713,38.380966,37.618645,38.754116,...,54.166538,65.598885,72.703720,37.245495,74.195480,50.303165,70.091690,51.561184,78.268379,"[193, 192, 193, 194, 194, 194, 193, 192, 168, ..."
4,66.725304,39.621262,32.244808,38.042030,58.565891,39.621262,72.515930,39.884468,36.982380,39.094852,...,64.889519,60.671410,77.523239,31.191755,76.997299,44.962749,73.707390,44.227142,86.871162,"[147, 148, 160, 196, 215, 214, 216, 217, 219, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7044,67.402550,31.842550,29.746750,38.632942,NaN,NaN,NaN,NaN,NaN,NaN,...,67.029091,NaN,NaN,NaN,NaN,NaN,NaN,50.426636,79.683922,"[71, 74, 85, 105, 116, 128, 139, 150, 170, 187..."
7045,66.134399,38.365501,30.478626,39.950199,NaN,NaN,NaN,NaN,NaN,NaN,...,66.626015,NaN,NaN,NaN,NaN,NaN,NaN,50.287395,77.983025,"[60, 60, 62, 57, 55, 51, 49, 48, 50, 53, 56, 5..."
7046,66.690735,36.845222,31.666420,39.685043,NaN,NaN,NaN,NaN,NaN,NaN,...,67.515160,NaN,NaN,NaN,NaN,NaN,NaN,49.462570,78.117119,"[74, 74, 74, 78, 79, 79, 79, 81, 77, 78, 80, 7..."
7047,70.965080,39.853664,30.543285,40.772339,NaN,NaN,NaN,NaN,NaN,NaN,...,66.724991,NaN,NaN,NaN,NaN,NaN,NaN,50.065186,79.586449,"[254, 254, 254, 254, 254, 238, 193, 145, 121, ..."


## Outlier Identification
As we saw in the EDA, there are a variety of types of images with a variety of keypoints. Below, we will remove some of the images we saw as outliers in the EDA.

Outlier types:
- Mislabelled images
- Weird/bad images
- All outliers (i.e. images that contain keypoints that are greater than 3 standard deviations away from the mean for that keypoint)

In [9]:
#CODE CELL FOR JOANIE

#This block is identifying and counting all outliers
#Outliers are images that contain keypoints > 3std from mean

train=train_data.drop(["image"],axis=1)
described_train=train.describe().T
std=described_train["std"]
mean=described_train["mean"]
q1=described_train["25%"]
q3=described_train["75%"]
iqr=q3-q1

#If we define outliers using IQR
#outlier_low=q1-1.5*iqr
#outlier_high=q3+1.5*iqr

#If we define outliers using std
outlier_low=mean-3*std
outlier_high=mean+3*std



#Keep track of these images in a list
outlier_images=[]
outlier_dict={}

#Iterate through the data to find outliers based on whether they are lower/higher than defined outlier boundaries
for col in train.columns:
    indices=list(np.where((train[col] < outlier_low[col]) | (train[col] > outlier_high[col]))[0])
    outlier_images.extend(indices)
    for i in indices:
        temp=outlier_dict.get(i,[])
        temp.append(col[:-1])
        outlier_dict[i]=temp

#Only count each index once
outliers=np.unique(outlier_images)
outliers
print("Finding points 3 standard deviations away from the mean results in ",len(outliers),
      "images being classified as outliers")
print("This represents",len(outliers)/train.shape[0]*100,"% of our total data")

print(train.shape)



Finding points 3 standard deviations away from the mean results in  409 images being classified as outliers
This represents 5.802241452688325 % of our total data
(7049, 30)


In [11]:
#CODE CELL FOR JOANIE

#This block is removing the worst outliers
#The worst outliers are the mislabelled images and the weird/bad images

miss_labelled = [1747, 1877, 1907,2199] #These are the images with keypoints that are not right
bad_images = [6492,6493,2430,3697] #These are the two collages and the two cartoons

worst_outliers = miss_labelled + bad_images

#Drop with inplace drops inplace
train_data.drop(index=worst_outliers,inplace=True)
train_data.shape

(7041, 31)

In [12]:
#CODE CELL FOR JOANIE

#This block is for removing all outliers as defined above

train_data.drop(index=outliers,inplace=True)

KeyError: '[1747 1877 1907 2199 2430 3697 6492 6493] not found in axis'